# Investigating K-fold Cross Validation

# TOC
___
[Sect.1: Introduction](#Sect.1)

[Sect.2: Linear Regression Model](#Sect.2)

[Sect.3: K-fold Cross Validation](#Sect.3)

[Sect.4: Comparison with Leave One Out Validation](#Sect.4)

# Sect.1
# Introduction
[Back to top](#TOC)

# Sect.2
# Linear Regression Model
[Back to top](#TOC)

# Sect.3
#  K-fold Cross Validation
[Back to top](#TOC)

# Sect.4
# Comparison with Leave One Out Validation
[Back to top](#TOC)